In [1]:
import collections
import os
import tempfile

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import urllib

import tensorflow as tf
import cv2

DatasetInfo = collections.namedtuple(
    'DatasetInfo',
    'num_classes, label_divisor, thing_list, colormap, class_names')


def _cityscapes_label_colormap():
    """Creates a label colormap used in CITYSCAPES segmentation benchmark.

    See more about CITYSCAPES dataset at https://www.cityscapes-dataset.com/
    M. Cordts, et al. "The Cityscapes Dataset for Semantic Urban Scene Understanding." CVPR. 2016.

    Returns:
      A 2-D numpy array with each row being mapped RGB color (in uint8 range).
    """
    colormap = np.zeros((256, 3), dtype=np.uint8)
    colormap[0] = [128, 64, 128]
    colormap[1] = [244, 35, 232]
    colormap[2] = [70, 70, 70]
    colormap[3] = [102, 102, 156]
    colormap[4] = [190, 153, 153]
    colormap[5] = [153, 153, 153]
    colormap[6] = [250, 170, 30]
    colormap[7] = [220, 220, 0]
    colormap[8] = [107, 142, 35]
    colormap[9] = [152, 251, 152]
    colormap[10] = [70, 130, 180]
    colormap[11] = [220, 20, 60]
    colormap[12] = [255, 0, 0]
    colormap[13] = [0, 0, 142]
    colormap[14] = [0, 0, 70]
    colormap[15] = [0, 60, 100]
    colormap[16] = [0, 80, 100]
    colormap[17] = [0, 0, 230]
    colormap[18] = [119, 11, 32]
    return colormap


def _cityscapes_class_names():
    return ('road', 'sidewalk', 'building', 'wall', 'fence', 'pole',
            'traffic_light', 'traffic_sign', 'vegetation', 'terrain', 'sky',
            'person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
            'bicycle')


def cityscapes_dataset_information():
    return DatasetInfo(
        num_classes=19,
        label_divisor=1000,
        thing_list=tuple(range(11, 19)),
        colormap=_cityscapes_label_colormap(),
        class_names=_cityscapes_class_names())

def perturb_color(color, noise, used_colors, max_trials=50, random_state=None):
    """Pertrubs the color with some noise.

    If `used_colors` is not None, we will return the color that has
    not appeared before in it.

    Args:
      color: A numpy array with three elements [R, G, B].
      noise: Integer, specifying the amount of perturbing noise (in uint8 range).
      used_colors: A set, used to keep track of used colors.
      max_trials: An integer, maximum trials to generate random color.
      random_state: An optional np.random.RandomState. If passed, will be used to
        generate random numbers.

    Returns:
      A perturbed color that has not appeared in used_colors.
    """
    if random_state is None:
        random_state = np.random

    for _ in range(max_trials):
        random_color = color + random_state.randint(
            low=-noise, high=noise + 1, size=3)
        random_color = np.clip(random_color, 0, 255)

        if tuple(random_color) not in used_colors:
            used_colors.add(tuple(random_color))
            return random_color

    print('Max trial reached and duplicate color will be used. Please consider '
          'increase noise in `perturb_color()`.')
    return random_color

def color_panoptic_map(panoptic_prediction, dataset_info, perturb_noise,output,select_object):
    """Helper method to colorize output panoptic map.

    Args:
      panoptic_prediction: A 2D numpy array, panoptic prediction from deeplab
        model.
      dataset_info: A DatasetInfo object, dataset associated to the model.
      perturb_noise: Integer, the amount of noise (in uint8 range) added to each
        instance of the same semantic class.

    Returns:
      colored_panoptic_map: A 3D numpy array with last dimension of 3, colored
        panoptic prediction map.
      used_colors: A dictionary mapping semantic_ids to a set of colors used
        in `colored_panoptic_map`.
    """
    if panoptic_prediction.ndim != 2:
        raise ValueError('Expect 2-D panoptic prediction. Got {}'.format(
            panoptic_prediction.shape))

    semantic_map = panoptic_prediction // dataset_info.label_divisor
    instance_map = panoptic_prediction % dataset_info.label_divisor
    height, width = panoptic_prediction.shape
    colored_panoptic_map = np.zeros((height, width, 3), dtype=np.uint8)

    used_colors = collections.defaultdict(set)
    # Use a fixed seed to reproduce the same visualization.
    random_state = np.random.RandomState(0)

    unique_semantic_ids = np.unique(semantic_map)
    semantic_choice=[]
    #print(_cityscapes_class_names())

    #0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
    choice_object = select_object #입력 데이터.
    frame_all_object = list(unique_semantic_ids)
    intersection = list(set(choice_object) & set(frame_all_object))
    # print(unique_semantic_ids)
    # print(unique_semantic_ids[1])
    # print(semantic_choice)
    # 원하는 부분의 슬라이싱 데이터를 새로운 배열에 넣고, 해당 부분을 하단의 for문에 넣어야함.

    bef_color_save_buf=[]
    aft_color_save_buf=[]

    color_map_list=[]
    score=0
    instance_score_buf=[]

    score_list=[] #<각 object 별 score>
    class_list =[] #<현 이미지에서 존재하는 object 종류>
    count_list=[] # <현 이미지에 존재하는 object의 종류별 개수>

    for semantic_id in intersection:#****객체분리 가능***배열 슬라이싱
        zero_list = np.zeros((height, width, 3), dtype=np.uint8)
        one_list = np.zeros((height, width, 3), dtype=np.uint8)
        # 정확도를 차집합으로 객체별 분리작업을 진행.    
        bef_color_save_buf = colored_panoptic_map.copy()
    #     print("befor colored_panoptic_map : ",np.unique(colored_panoptic_map))
    #     print("0befor bef_color_save_buf : ",np.unique(bef_color_save_buf))
        ##########################################
        semantic_mask = semantic_map == semantic_id #semantic_id와 map이 같이면 True
        if semantic_id in dataset_info.thing_list:
            # For `thing` class, we will add a small amount of random noise to its
            # correspondingly predefined semantic segmentation colormap.
            unique_instance_ids = np.unique(instance_map[semantic_mask])
            for instance_id in unique_instance_ids:
                instance_mask = np.logical_and(semantic_mask,instance_map == instance_id)
                random_color = perturb_color(
                        dataset_info.colormap[semantic_id],
                        perturb_noise,
                        used_colors[semantic_id],
                        random_state=random_state)
                colored_panoptic_map[instance_mask] = random_color
        else:
          # For `stuff` class, we use the defined semantic color.
            colored_panoptic_map[semantic_mask] = dataset_info.colormap[semantic_id]
            used_colors[semantic_id].add(tuple(dataset_info.colormap[semantic_id]))

        aft_color_save_buf = colored_panoptic_map.copy()
    #     print("befor buf : ",np.unique(bef_color_save_buf))
    #     print("after buf : ",np.unique(aft_color_save_buf))
        #이전 color map과 이후 color map이 동일할 경우 after의 값을 넣고 아니면 0을 넣음
        #able_color는 객체가 새로 갱신될때마다 리셋됨.
        #완성된 able_color를 저장해놀 list가 필요함. class의 개수만큼 늘어나야함.

    #     print(np.array_equal(bef_color_save_buf[0][0],aft_color_save_buf[0][0]))
         # list의 1차원은 세로 2차원은 가로 픽셀을 의미함.3차원은 RGB
        #zero_list = np.where(np.array_equal(bef_color_save_buf, aft_color_save_buf), [0,0,0], aft_color_save_buf)
        #color_map_list.append(zero_list)
    #     zero_list = np.zeros_like(aft_color_save_buf)
    #     zero_list[np.all(bef_color_save_buf == aft_color_save_buf, axis=-1)] = [0,0,0]
    #     zero_list[np.all(bef_color_save_buf != aft_color_save_buf, axis=-1)] = aft_color_save_buf[np.all(bef_color_save_buf != aft_color_save_buf, axis=-1)]
    #     color_map_list.append(zero_list)

        for x in range(len(aft_color_save_buf)):
            for y in range(len(aft_color_save_buf[x])):
                if np.array_equal(bef_color_save_buf[x][y],aft_color_save_buf[x][y]):
                    zero_list[x][y] = [0,0,0]
                else :
                    zero_list[x][y] = aft_color_save_buf[x][y]
        color_map_list.append(zero_list)

        #print("colored_panoptic_map :",colored_panoptic_map)
        #print("colored_panoptic_map의 shape : ",colored_panoptic_map.shape)
        #instance,sementic score
        #print(np.shape(output['instance_scores'][0])) #픽셀별 instance score
        #print(np.shape(output['semantic_scores'][0])) #픽셀별 semantic score

        # 정확도는 insance와 semantic 구분을 해야함, thing의 경우 instance stuff의 경우 semantic의 정확도를 사용하면 될것으로 보임
        #thing_list=tuple(range(11, 19))
        # 구분방법은 어떻게?

        object_list=_cityscapes_class_names()

        if semantic_id<11 :
            one_list = np.where(zero_list>=1,1,zero_list)
            instance_score_buf = np.array(output['semantic_scores'][0]).reshape((height,width,1))
            instance_score_buf=one_list*instance_score_buf
            score = instance_score_buf[(instance_score_buf!=0)].mean()
            score_list.append(score)
            #print(f"semant score : {object_list[semantic_id]} : ",score)      
        else :
            one_list = np.where(zero_list>=1,1,zero_list)
            instance_score_buf = np.array(output['instance_scores'][0]).reshape((height,width,1))
            instance_score_buf=one_list*instance_score_buf
            score = instance_score_buf[(instance_score_buf!=0)].mean()
            score_list.append(score)
            #print(f"inst score : {object_list[semantic_id]} : ",score)       



        #print(f"used_colors: ",used_colors)
    for key in used_colors: 
        #print(key)
        if(key!=255):
            class_list.append(object_list[key])
            count_list.append(len(used_colors[key]))
            #print(f"key={object_list[key]}는 {len(used_colors[key])}개 이다.")

        #     print("for=======================")
    panoptic_map=colored_panoptic_map
        #plt.imshow(panoptic_map)
        # print("==================================")
        # print(colored_panoptic_map)
        # print(used_colors)
    #print("one : ",colored_panoptic_map,used_colors,class_list,count_list,score_list)
    return colored_panoptic_map,used_colors,class_list,count_list,score_list
    
    
def vis_segmentation(image,
                     panoptic_prediction,
                     dataset_info,
                     output,select_object,
                     perturb_noise=60
                     ):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(30, 20))
    grid_spec = gridspec.GridSpec(2, 2)

    ax = plt.subplot(grid_spec[0])
    #plt.imshow(image)
    plt.axis('off')
    ax.set_title('input image', fontsize=20)

    ax = plt.subplot(grid_spec[1])
    panoptic_map, used_colors,class_list,count_list,score_list = color_panoptic_map(panoptic_prediction,
                                                  dataset_info, perturb_noise,output,select_object)
    #print("map panoptic_map",panoptic_map)
    #print("map used_colors",used_colors)
    #print("map class_list",class_list)
    #print("map count_list",count_list)
    #print("map score_list",score_list)
    
    #plt.imshow(panoptic_map)
    plt.axis('off')
    ax.set_title('panoptic map', fontsize=20)

    ax = plt.subplot(grid_spec[2])
    #plt.imshow(image)
    #plt.imshow(panoptic_map, alpha=0.7)
    plt.axis('off')
    ax.set_title('panoptic overlay', fontsize=20)

    ax = plt.subplot(grid_spec[3])
    try :
        max_num_instances = max(len(color) for color in used_colors.values())
        # RGBA image as legend.
        legend = np.zeros((len(used_colors), max_num_instances, 4), dtype=np.uint8)
        class_names = []
        for i, semantic_id in enumerate(sorted(used_colors)):
            legend[i, :len(used_colors[semantic_id]), :3] = np.array(
                list(used_colors[semantic_id]))
            legend[i, :len(used_colors[semantic_id]), 3] = 255
            if semantic_id < dataset_info.num_classes:
                class_names.append(dataset_info.class_names[semantic_id])
            else:
                class_names.append('ignore')
        #plt.imshow(legend, interpolation='nearest')
        ax.yaxis.tick_left()
        plt.yticks(range(len(legend)), class_names, fontsize=15)
        plt.xticks([], [])
        ax.tick_params(width=0.0, grid_linewidth=0.0)
        plt.grid('off')
        #plt.show()
        #print(class_list,count_list,score_list)
    except :
        print("non object")
    return panoptic_map, used_colors,class_list,count_list,score_list
_MODELS = ('resnet50_os32_panoptic_deeplab_cityscapes_crowd_trainfine_saved_model',#13초 가량 소요
           'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model', # 7초 가량 소요(성능이 좋지 않음)
           'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', # 26초 소요
           'swidernet_sac_1_1_1_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_3_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_4.5_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_1_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_3_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_4.5_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_s_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_l_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'resnet50_kmax_deeplab_cityscapes_train',
           'axial_resnet50_kmax_deeplab_cityscapes_train',
           'convnext_tiny_kmax_deeplab_cityscapes_train',
           'convnext_small_kmax_deeplab_cityscapes_train',
           'convnext_base_kmax_deeplab_cityscapes_train',
           'convnext_large_kmax_deeplab_cityscapes_train',)

In [2]:
#압축해제용 블럭
# for x in _MODELS :
#     MODEL_NAME= x
#     _DOWNLOAD_URL_PATTERN = 'https://storage.googleapis.com/gresearch/tf-deeplab/saved_model/%s.tar.gz'

#     _MODEL_NAME_TO_URL_AND_DATASET = {
#         model: (_DOWNLOAD_URL_PATTERN % model, cityscapes_dataset_information())
#         for model in _MODELS
#     }

#     MODEL_URL, DATASET_INFO = _MODEL_NAME_TO_URL_AND_DATASET[MODEL_NAME]
#     ####################################################################
#     model_dir = "./model_dum/"
#     print(model_dir)

#     download_path = os.path.join(model_dir, MODEL_NAME + '.gz')
#     print(download_path)

#     urllib.request.urlretrieve(MODEL_URL, download_path)
#     print('저장')
#     !tar -xzvf {download_path} -C {model_dir}
#     ####################################################################
#     LOADED_MODEL = tf.saved_model.load(os.path.join(model_dir, MODEL_NAME))
#     ####################################################################
#     count=0
#     avg_timer = 0

In [3]:
#모델선택용 블럭

MODEL_NAME= _MODELS[11]
_DOWNLOAD_URL_PATTERN = 'https://storage.googleapis.com/gresearch/tf-deeplab/saved_model/%s.tar.gz'

_MODEL_NAME_TO_URL_AND_DATASET = {
    model: (_DOWNLOAD_URL_PATTERN % model, cityscapes_dataset_information())
    for model in _MODELS
}

MODEL_URL, DATASET_INFO = _MODEL_NAME_TO_URL_AND_DATASET[MODEL_NAME]
####################################################################
model_dir = "./model_dum/"
####################################################################
LOADED_MODEL = tf.saved_model.load(os.path.join(model_dir, MODEL_NAME))
####################################################################
count=0
avg_timer = 0

In [ ]:
#frame 가져오기
#1개씩 가져오자.
#50004포트로 frame 하나만 가져오자.
import socket
import struct
import pickle
import cv2
import time
from PIL import Image
import io

#변수
data_buffer = b"" #바이너리형태 버퍼
data_size = struct.calcsize("L") 

list_recv_socket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
list_recv_socket.bind(('172.30.1.52',50006))
print('list 바인드 완료')
list_recv_socket.listen(10)
print('list 대기 완료')
list_recv_socket.setblocking(0)
list_recv_socket.settimeout(0.1)
print('list 셋팅 완료')

###########Panoptic분석결과가 출력될 경우 toggle하는 socket############
ip2 = '172.30.1.52'
port2 = 50003
tg_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #소켓생성
tg_socket.connect((ip2,port2)) #발신소켓연결
select_object = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
byte_select_object=bytes(select_object)

#############frame소켓
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #소켓생성
server_socket.bind(('172.30.1.52', 50004)) #수신소켓연결
server_socket.listen(10) #Frame 수신대기 10개 소켓까지 가능.
print("연결요청 대기중")
client_socket, addr = server_socket.accept()  #Frame 연결 요청을 수락함. 그러면 아이피주소, 포트등 데이터를 return
print("연결요청 수락")

#############결과 데이터 송출########
result_data_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_data_socket.connect(('172.30.1.63',50011)) #지수 컴퓨터로 보내기
result_data_socket2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_data_socket2.connect(('172.30.1.88',50013)) #테스트를 위해 내 컴퓨터로 보내기
result_data_socket3 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_data_socket3.connect(('172.30.1.6',50020)) #지수 컴퓨터로 보내기
#############결과 이미지 송출########
result_img_data_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_img_data_socket.connect(('172.30.1.63',50012)) #지수 컴퓨터로 보내기
result_img_data_socket2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_img_data_socket2.connect(('172.30.1.88',50014)) #테스트를 위해 내 컴퓨터로 보내기
result_img_data_socket3 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
result_img_data_socket3.connect(('172.30.1.6',50021)) #지수 컴퓨터로 보내기

byte_list_data = b'0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19'
while True:
    ##########Selcet_object_List_recv_socket############
    try :
        list_socket_ct, addr = list_recv_socket.accept()
        byte_list_data = list_socket_ct.recv(1024)
        #str_select_object = byte_select_object.decode()
        #print("byte_select_object",byte_select_object)
        #select_object = list(byte_select_object) #data크기문제.
    except socket.timeout:
        byte_list_data = byte_list_data
    
    
    if byte_list_data!=b'':
        int_list = list(map(int,byte_list_data.decode().split(",")))
    #print("int_list",int_list)
    select_object = int_list
    
    ###########Frame 1장을 수신하는 socket############    
##
    #################frame data수신
    while len(data_buffer) < data_size:
        data_buffer += client_socket.recv(65536)     # Frame 데이터를 받음. 출력되는 버퍼 사이즈.
    packed_data_size = data_buffer[:data_size]
    data_buffer = data_buffer[data_size:] 
    frame_size = struct.unpack(">L", packed_data_size)[0]
    
    while len(data_buffer) < frame_size:
        data_buffer += client_socket.recv(65536)
        print(data_buffer)
    frame_data = data_buffer[:frame_size]
    data_buffer = data_buffer[frame_size:]
    
    #print("수신 프레임 크기 : {} bytes".format(frame_size))
    ###########################################################
    #print("frame_data",frame_data)
    #frame 직렬화 등. 디코드 진행.
    frame = pickle.loads(frame_data) 
    frame = cv2.imdecode(frame,cv2.COLOR_BGR2RGB)
    
    #panoptic_segmentation 진행.
    output = LOADED_MODEL(tf.cast(frame, tf.uint8))
    panoptic_map, used_colors,class_list,count_list,score_list = vis_segmentation(frame, output['panoptic_pred'][0], DATASET_INFO,output,select_object)   
    #print("panoptic연산완료")
    
    #소켓 연산데이터 전송
    #print("vispanoptic_map",panoptic_map)
    #print("visused_colors",used_colors)
    class_string = " ".join(class_list)
    count_string = " ".join(map(str,count_list))
    score_string = " ".join(map(str,score_list))
    #print("class_string: "+class_string)
    #print("count_string: "+count_string)
    #print("score_string: "+score_string)

    all_string = class_string +"/"+count_string +"/" + score_string
    #print("all_string: " + all_string)
    #print(type(all_string))
    allsend = all_string.encode('utf-8')

    result_data_socket.sendall(allsend)
    result_data_socket2.sendall(allsend)
    result_data_socket3.sendall(allsend)
    #print("socket전송 결과데이터 : ",allsend)
    
    #소켓 결과 이미지 전송
#     #print("panoptic_map",panoptic_map)
#     res_retval, res_frame = cv2.imencode('.jpg', panoptic_map, [cv2.IMWRITE_JPEG_QUALITY, 80])
#     #print("res_frame1",res_frame)
#     res_frame = pickle.dumps(res_frame)
#     #print("res_frame2",res_frame)
#     result_img_data_socket.sendall(struct.pack(">L", len(res_frame)) + res_frame)
    
    result_img = Image.fromarray(panoptic_map)
    b = io.BytesIO()
    result_img.save(b, 'jpeg')
    result_bytes=b.getvalue()
    #result_img_data_socket.sendall(struct.pack(">L", len(result_bytes)) + result_bytes)
    result_img_data_socket.send(result_bytes)
    result_img_data_socket2.send(result_bytes)
    result_img_data_socket3.send(result_bytes)
    #result_img_data_socket.send(result_bytes,('172.30.1.52',50007))
    
    #연산이 완료되었으니 토글 전송
    tg_socket.sendall(struct.pack(">L", 1) + b'1')
    
    # q 키 입력시 종료(미사용중)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
# 소켓 닫기
client_socket.close()
server_socket.close()
print('연결 종료')

# 모든 창 닫기
cv2.destroyAllWindows()
  

list 바인드 완료
list 대기 완료
list 셋팅 완료
연결요청 대기중
연결요청 수락


C:\Users\smhrd\anaconda3\envs\panoptic_topdown\lib\site-packages\ipykernel_launcher.py:256: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object
non object

In [ ]:
LOADED_MODEL

In [ ]:
# #모델 시간측정
# model_time ={}
# start=time.time()
# output = LOADED_MODEL(tf.cast(frame, tf.uint8))
# vis_segmentation(frame, output['panoptic_pred'][0], DATASET_INFO)   
# #####
# count += 1
# timer = time.time() - start
# avg_timer += timer
# print("time :", timer)
# print(count)